Step 1: Setting Up the Spark Environment

 A. Deploy a Spark Cluster (like AWS EMR, GCP Dataproc, or an on-prem Hadoop
cluster, Azure HD Insight).
 
 B. Store Data in HDFS instead of local storage.
• Load data from Kaggle i.e. Data Source (#!/bin/bash curl -L -o ~/olist/brazilianecommerce.zip\ https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilianecommerce)
• !unzip brazilian-ecommerce.zip -d ~/olist/data/

C. Use PySpark to interact with data.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName('OlistData') \
.getOrCreate()


25/03/26 14:27:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

In [3]:
!hadoop fs -ls /data/olist

Found 9 items
-rw-r--r--   2 beingabhisheksahu hadoop    9033957 2025-03-25 12:35 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 beingabhisheksahu hadoop   61273883 2025-03-25 12:35 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 beingabhisheksahu hadoop   15438671 2025-03-25 12:35 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 beingabhisheksahu hadoop    5777138 2025-03-25 12:35 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 beingabhisheksahu hadoop   14451670 2025-03-25 12:35 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 beingabhisheksahu hadoop   17654914 2025-03-25 12:35 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 beingabhisheksahu hadoop    2379446 2025-03-25 12:35 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 beingabhisheksahu hadoop     174703 2025-03-25 12:35 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 beingabhisheksahu hadoop       2613 2025-03-25 12:35 /data/olist/product_category_name_translation.c

In [4]:
hdfs_path = '/data/olist/'

In [ ]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header = True,inferSchema = True)

25/03/26 14:46:18 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [8]:
customers_df.show(5)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



In [10]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
order_item_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path +'product_category_name_translation.csv',header=True,inferSchema=True)


In [11]:
orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [12]:
# Data Lekage or Drop
print(f'Customers : {customers_df.count()} rows')
print(f'Orders : {orders_df.count()} rows')

Customers : 99441 rows
Orders : 99441 rows


In [13]:
customers_df.columns

['customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']

In [15]:
from pyspark.sql.functions import col,when ,count

# Check for nulls in critical fields
customers_df.select([count(when(col(c).isNull(),1)).alias(c) for c in customers_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [16]:
# Duplicate Values
customers_df.groupBy('customer_id').count().filter('count>1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [17]:
# Customer Distribution by state

customers_df.groupBy('customer_state').count().orderBy('count',ascending = False).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [18]:
payments_df.groupBy('payment_type').count().orderBy('count',ascending=False).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [19]:
from pyspark.sql.functions import sum
top_products = order_item_df.groupBy('product_id').agg(sum('price').alias('total_sales'))
top_products.orderBy('total_sales',ascending=False).show(20)

+--------------------+------------------+
|          product_id|       total_sales|
+--------------------+------------------+
|bb50f2e236e5eea01...|           63885.0|
|6cdd53843498f9289...| 54730.20000000005|
|d6160fb7873f18409...|48899.340000000004|
|d1c427060a0f73f6b...| 47214.51000000006|
|99a4788cb24856965...|43025.560000000085|
|3dd2a17168ec895c7...| 41082.60000000005|
|25c38557cf793876c...| 38907.32000000001|
|5f504b3a1c75b73d6...|37733.899999999994|
|53b36df67ebb7c415...| 37683.42000000001|
|aca2eb7d00ea1a7b8...| 37608.90000000007|
|e0d64dcfaa3b6db5c...|          31786.82|
|d285360f29ac7fd97...|31623.809999999983|
|7a10781637204d8d1...|           30467.5|
|f1c7f353075ce59d8...|          29997.36|
|f819f0c84a64f02d3...|29024.479999999996|
|588531f8ec37e7d5f...|28291.989999999998|
|422879e10f4668299...|26577.219999999972|
|16c4e87b98a9370a9...|           25034.0|
|5a848e4ab52fd5445...|24229.029999999962|
|a62e25e09e05e6faf...|           24051.0|
+--------------------+------------

In [20]:
# Average Delivery Time Analysis
delivery_df = orders_df.select('order_id','order_purchase_timestamp','order_delivered_customer_date')
delivery_df.show()

+--------------------+------------------------+-----------------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|
+--------------------+------------------------+-----------------------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|
|a4591c265e18cb1dc...|     2017-07-09 21:57:05|          2017-07-26 10:57:55|
|136cce7faa42fdb2c...|     2017-04-11 12:22:08|                         NULL|
|6514b8ad8028c9f2c...|     2017-05-16 13:10:30|          2017-05-26 12:55:51|
|76c6e866289321a7c...|     2017-01-23 18:29:09|          2017-02-02 14:08:10|
|e69bfb5eb88e0ed6a...|     2017-07-29 11:55:02|          2017-08

In [24]:
from pyspark.sql.functions import datediff,to_date
delivery_detail_df = delivery_df.withColumn('delivery_time',datediff(col('order_delivered_customer_date'),col('order_purchase_timestamp')))
delivery_detail_df.show()

+--------------------+------------------------+-----------------------------+-------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|delivery_time|
+--------------------+------------------------+-----------------------------+-------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|            8|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|           14|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|            9|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|           14|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|            3|
|a4591c265e18cb1dc...|     2017-07-09 21:57:05|          2017-07-26 10:57:55|           17|
|136cce7faa42fdb2c...|     2017-04-11 12:22:08|                         NULL|         NULL|
|6514b8ad8028c9f2c...|     2017-05-16 13:10:30|          2017-05-26 12:55:51|   

In [25]:
delivery_detail_df.orderBy('delivery_time',ascending=False).show()

+--------------------+------------------------+-----------------------------+-------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|delivery_time|
+--------------------+------------------------+-----------------------------+-------------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|          210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|          208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|          196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|          195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|          195|
|0f4519c5f1c541dde...|     2017-03-09 13:26:57|          2017-09-19 14:38:21|          194|
|47b40429ed8cce3ae...|     2018-01-03 09:44:01|          2018-07-13 20:51:31|          191|
|2fe324febf907e3ea...|     2017-03-13 20:17:10|          2017-09-19 17:00:07|   

In [26]:
 # Identify Missing Values
    
from pyspark.sql.functions import *
def missing_values(df,df_name):
   print(f'Missing values in {df_name}:')
   df.select([count(when(col(c).isNull(),1)).alias(c) for c in df.columns]).show()

In [27]:
missing_values(customers_df,'customer')

Missing values in customer:


+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



2 Handle Missing Values
 1. Drop missing Values ( for non- critical columns )
 2. Fill missing values ( for numerical columns )
 3. Impute Missing Values ( for continous data )

In [28]:
 orders_df_cleaned = orders_df.na.drop(subset=['order_id','customer_id','order_status'])
    
 orders_df_cleaned.show()  

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [29]:
orders_df_cleaned =orders_df.fillna({'order_delivered_customer_date':'9999-12-31'})
orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [31]:
payments_df_with_null = payments_df.withColumn('payment_value',when(col('payment_value')!= 99.33,col('payment_value')).otherwise(lit(None)))
payments_df_with_null.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [32]:
 from pyspark.ml.feature import Imputer
 imputer =Imputer(inputCols=['payment_value'],outputCols=['payment_value_imputed']).setStrategy('median')
 payments_df_cleaned = imputer.fit(payments_df_with_null).transform(payments_df_with_null)
 payments_df_cleaned.show()

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payment_value_imputed|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|                100.0|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|               

 Standardizing the format

In [33]:
 def print_schema(df,df_name):
  print(f'schema of {df_name}:')
  df.printSchema()

In [34]:
print_schema(orders_df,'orders')

schema of orders:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [35]:
orders_df_cleaned = orders_df_cleaned.withColumn('order_purchase_timestamp',to_date(col('order_purchase_timestamp')))\
                                    .withColumn('order_approved_at',to_date(col('order_approved_at')))
orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|              2017-10-02|       2017-10-02|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|              2018-07-24|       2018-07-26|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered

In [36]:
payments_df_condition= payments_df_cleaned.withColumn('payment_type',when(col('payment_type')=='boleto','BankTransfer')
 .when(col('payment_type')=='credit_card','CreditCard')
 .when(col('payment_type')=='debit_card','DebitCard')
 .otherwise('other'))

payments_df_condition.show()

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payment_value_imputed|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1|  CreditCard|                   8|         NULL|                100.0|
|a9810da82917af2d9...|                 1|  CreditCard|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1|  CreditCard|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1|  CreditCard|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1|  CreditCard|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1|  CreditCard|                   2|        96.12|               

In [37]:
customers_df_cleaned = customers_df.withColumn('customer_zip_code_prefix',col('customer_zip_code_prefix').cast('string'))
customers_df_cleaned.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



***Remove Duplicate Records***

In [39]:
customers_df_cleaned = customers_df_cleaned.dropDuplicates(['customer_id'])

In [40]:
 order_with_details = orders_df_cleaned.join(order_item_df,'order_id','left')\
 .join(payments_df_cleaned,'order_id','left')\
 .join(customers_df_cleaned,'customer_id','left')

In [43]:
order_with_details.show()

+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+------+-------------+------------------+------------+--------------------+-------------+---------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|            order_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|payment_sequential|payment_type|payment_installments|payment_value|payment_value_imputed|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------+------------------------+-----

In [44]:
order_with_total_value = order_with_details.groupBy('order_id')\
 .agg(sum('payment_value').alias('total_order_value')).orderBy('total_order_value',ascending=False)

In [45]:
order_with_total_value.show()

+--------------------+------------------+
|            order_id| total_order_value|
+--------------------+------------------+
|03caa2c082116e1d3...|         109312.64|
|ab14fdcfbe524636d...| 45256.00000000001|
|1b15974a0141d54e3...|44048.000000000015|
|2cc9089445046817a...|          36489.24|
|e8fa22c3673b1dd17...|30185.999999999993|
|736e1922ae60d0d6a...|          29099.52|
|9aec4e1ae90b23c7b...|           22346.6|
|71dab1155600756af...|21874.049999999996|
|912343626f370ead5...|          19457.04|
|4412d97cb2093633a...|          19174.38|
|9f5054bd9a3c71702...|18667.000000000004|
|428a2f660dc84138d...|          18384.75|
|3a213fcdfe7d98be7...|          17786.88|
|f80549a97eb203e15...|           17671.0|
|cf4659487be50c0c3...|          17069.76|
|be382a9e1ed251281...|16313.600000000002|
|637617b3ffe9e2f7a...|14963.639999999998|
|c52c7fbe316b5b9d5...|14577.569999999998|
|f60ce04ff8060152c...|14401.000000000002|
|73c8ab38f07dc9438...|14196.280000000004|
+--------------------+------------

**Advance Transformation**

In [47]:
quantiles = order_item_df.approxQuantile('price',[0.01,0.99],0.0)
low_cutoff,high_cutoff = quantiles[0],quantiles[1]

In [48]:
low_cutoff,high_cutoff

(9.99, 890.0)

In [49]:
order_item_df.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            112650|
|   mean|120.65373901471354|
| stddev|183.63392805026012|
|    min|              0.85|
|    25%|              39.9|
|    50%|             74.99|
|    75%|             134.9|
|    max|            6735.0|
+-------+------------------+



In [50]:
order_item_df_cleaned = order_item_df.filter((col('price') >=low_cutoff) & (col('price') <=high_cutoff))
order_item_df_cleaned.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [51]:
order_item_df_cleaned.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            110453|
|   mean|108.49213068006871|
| stddev|112.87303173792675|
|    min|              9.99|
|    25%|             39.99|
|    50%|              74.9|
|    75%|             130.0|
|    max|             890.0|
+-------+------------------+



In [52]:
 products_df.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [53]:
products_df_cleaned= products_df.withColumn(
 'product_size_category',
 when(col('product_weight_g') <500,'Small')
 .when(col('product_weight_g').between(500,2000),'Medium')
 .otherwise('Large')
 )

In [54]:
products_df_cleaned.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_size_category|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|                Small|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|        

In [56]:
!hadoop fs -mkdir /data/olist_proc

In [57]:
order_with_details.write.mode('overwrite').parquet('/data/olist_proc/cleaned_data.parquet')
products_df_cleaned.write.mode('overwrite').parquet('/data/olist_proc/product_df_cleaned.parquet')

In [59]:
!hadoop fs -ls /data/olist_proc

Found 2 items
drwxr-xr-x   - root hadoop          0 2025-03-25 18:00 /data/olist_proc/cleaned_data.parquet
drwxr-xr-x   - root hadoop          0 2025-03-25 18:00 /data/olist_proc/product_df_cleaned.parquet


In [1]:
!hadoop fs -ls /data/olist_proc

Found 2 items
drwxr-xr-x   - root hadoop          0 2025-03-25 18:00 /data/olist_proc/cleaned_data.parquet
drwxr-xr-x   - root hadoop          0 2025-03-25 18:00 /data/olist_proc/product_df_cleaned.parquet


Optimized Joins For Data integration

In [ ]:
# Cache Frequently used Data for Better Performance
orders_df.cache()
customers_df.cache()
order_item_df.cache()

In [ ]:
from pyspark.sql.functions import *

orders_items_joined_df = orders_df.join(order_item_df,'order_id','inner')
orders_items_products_df = orders_items_joined_df.join(products_df,'product_id','inner')
orders_items_products_sellers_df = orders_items_products_df.join(broadcast(sellers_df),'seller_id','inner')
full_orders_df = orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

# GEolocation Data

full_orders_df = full_orders_df.join(broadcast(geolocation_df),full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')
full_orders_df = full_orders_df.join(broadcast(reviews_df),'order_id','left')
full_orders_df = full_orders_df.join(payments_df,'order_id','left')
full_orders_df.cache()


Window Function and Ranking

In [ ]:
from pyspark.sql.window import Window

# Dense Rank for Sellers Based on Revenue
window_spec = Window.partitionBy('seller_id').orderBy(desc('price'))

# Rank Top Selling Products Per seller

top_seller_products_df = full_orders_df.withColumn('rank',rank().over(window_spec)).filter(col('rank')<=5)
top_seller_products_df.select('seller_id','price','rank').show()